In [1]:
import ROOT

Welcome to JupyROOT 6.14/00


In [2]:
myfile = ROOT.TFile('/home/mikhasenko/cernbox/tmp/pwa_results/hfit_0.100000-0.112853.root')

In [7]:
############################################

def get_cve(name,i) :
    hist = myfile.Get(name)
    return [hist.GetBinCenter(i), hist.GetBinContent(i), hist.GetBinError(i)]

############################################

def get_SDM(bin) :
    SDM = [[0 for x in range(88)] for y in range(88)]
    eSDM = [[0 for x in range(88)] for y in range(88)]
    # zero epsilon
    for i in range(88) :
        v = get_cve('h'+str(i+1), bin)
        SDM[i][i] = v[1]
        eSDM[i][i] = v[2]
    # positive epsilon
    for i in range(1,81) :
        for j in range(1,81) :
            if (i >= j) : continue
            name = 'h1'+'{:03d}'.format(i+1)+'{:03d}'.format(j+1)
            vr = get_cve(name, bin)
            name = 'h2'+'{:03d}'.format(i+1)+'{:03d}'.format(j+1)
            vi = get_cve(name, bin)
            SDM[i][j] = vr[1]+vi[1]*1j
            eSDM[i][j] = v[2]+vi[2]*1j
            SDM[j][i] = vr[1]-vi[1]*1j
            eSDM[j][i] = v[2]-vi[2]*1j
    # negative epsilon
    for i in range(82,88) :
        for j in range(82,88) :
            if (i >= j) : continue
            name = 'h1'+'{:03d}'.format(i+1)+'{:03d}'.format(j+1)
            vr = get_cve(name, bin)
            name = 'h2'+'{:03d}'.format(i+1)+'{:03d}'.format(j+1)
            vi = get_cve(name, bin)
            SDM[i][j]  = vr[1]+vi[1]*1j
            eSDM[i][j] = vr[2]+vi[2]*1j
            SDM[j][i]  = vr[1]-vi[1]*1j
            eSDM[j][i] = vr[2]-vi[2]*1j
    return SDM, eSDM

############################################

import numpy as np
def save2file(output, inmat) :
    mat=np.matrix([np.array(v).real for v in inmat])
    np.savetxt(output+'.re',mat)
    mat=np.matrix([np.array(v).imag for v in inmat])
    np.savetxt(output+'.im',mat)


Get the matrixes and Save them

In [8]:
SDM91, eSDM91 = get_SDM(91)
save2file('/tmp/sdm91', SDM91)
save2file('/tmp/sdm91-err', eSDM91)